<a href="https://colab.research.google.com/github/pinnero/pinnero/blob/main/fresco_deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import torchvision.models as models
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define data transformations
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Update the paths to your data in Google Drive
drive_root = '/content/drive/MyDrive'  # Root directory of your Google Drive
data_path = f"{drive_root}/labeled_images"  # Update this path

# Load dataset and apply transformations
dataset = ImageFolder(root=data_path, transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders for training and validation
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [17]:
# Define your custom CNN architecture with ResNet transfer learning

class CustomCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(CustomCNN, self).__init__()
        # Load a pre-trained ResNet model (ResNet18 in this case)
        self.resnet = models.resnet18(pretrained=True)

        # Freeze all layers of ResNet to avoid training them
        for param in self.resnet.parameters():
            param.requires_grad = False

       # Replace the last fully connected layer of ResNet
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(in_features, num_classes)  # Change this line




    def forward(self, x):
        x = self.resnet(x)
        return x

# Initialize the custom CNN model
model = CustomCNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:

import matplotlib.pyplot as plt

# Lists to store accuracy values per epoch
val_accuracy_history = []

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation after each epoch
    model.eval()
    val_loss = 0.0
    correct = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass and compute loss
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()

            # Count correct predictions
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    accuracy = 100.0 * correct / len(val_dataset)

    # Append validation accuracy to history
    val_accuracy_history.append(accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}] | Validation Loss: {val_loss:.4f} | Accuracy: {accuracy:.2f}%")

# Plot the validation accuracy trends
epochs = list(range(1, num_epochs + 1))
plt.plot(epochs, val_accuracy_history)
plt.xlabel("Epochs")
plt.ylabel("Validation Accuracy (%)")
plt.title("Validation Accuracy per Epoch")
plt.show()


Epoch [1/10] | Validation Loss: 1.3839 | Accuracy: 30.51%
Epoch [2/10] | Validation Loss: 1.3303 | Accuracy: 38.98%
Epoch [3/10] | Validation Loss: 1.2856 | Accuracy: 40.68%
Epoch [4/10] | Validation Loss: 1.1965 | Accuracy: 49.15%


In [ ]:

test_path = "/path/to/your/test/dataset"  # Replace with the path to your test dataset
test_dataset = ImageFolder(root=test_path, transform=transform)

# Create the test_loader using DataLoader
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Training and validation loops as you provided before...
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        # Training loop as before...

    model.eval()
    val_loss = 0.0
    correct_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            # Validation loop as before...

    # Print validation results as before...

    # Test loop
    model.eval()
    test_loss = 0.0
    correct_test = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass for test data
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Count correct predictions for accuracy calculation
            _, predicted = torch.max(outputs.data, 1)
            correct_test += (predicted == labels).sum().item()

    # Calculate average test loss and accuracy
    test_loss /= len(test_loader)
    test_accuracy = 100.0 * correct_test / len(test_dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}] | Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.2f}%")

# Training and validation completed after all epochs...
print("Training and Testing completed.")
